In [ ]:
!pip uninstall -y kaggle
!pip install kaggle==1.5.6
%env KAGGLE_USERNAME = bassamsabber
%env KAGGLE_KEY = 717678f8ebfb53d84e9c47444c275871
!kaggle competitions download -c computationalintelligencesc2020
!unzip DataSet
!unzip computationalintelligencesc2020

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.regularizers import l2
import numpy as np
import matplotlib.pyplot as plt 
from keras.utils.vis_utils import plot_model
import pandas as pd
import os
from keras.layers.merge import concatenate
from sklearn import preprocessing




In [ ]:
trdata = ImageDataGenerator()g
traindata = trdata.flow_from_directory(directory="/content/Scenes training set/Scenes training set",target_size=(150,150))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="/content/Scenes testing test", target_size=(150,150))

In [ ]:
model = Sequential()
model.add(Conv2D(input_shape=(150,150,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.7))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dropout(0.7)) 
model.add(Dense(units=4096,activation="relu"))
model.add(Dropout(0.5)) 
model.add(Dense(units=6, activation="softmax"))

if os.path.exists('VGG16.h5'):
  classLabels = dict({0:'buildings',1:'forest',2:'glacier',3:'mountain',4:'sea',5:'street'})
  saved = load_model('VGG16.h5')
  from keras.preprocessing import image
  testFolder = r'/content/Scenes testing test/Scenes testing test'
  images=[]
  features=[]
  for img in os.listdir(testFolder):
    image_path = os.path.join(testFolder,img)
    images.append(img)
    img = image.load_img(image_path, target_size=(150,150))
    x = image.img_to_array(img)
    #x = np.expand_dims(x, axis = 0)
    features.append(x)
  features = np.array(features)
  images = np.array(images)
  output = saved.predict(features)
  maxIndex = np.argmax(output,axis=1)
  labels = []
  for i in maxIndex:
    labels.append(classLabels[i])
  
  submit = pd.read_csv('/content/submit.csv')
  submit['Image'] = images
  submit['Label'] = maxIndex
  submit.to_csv("submit.csv")
  print("Done!")

else:
  from keras.optimizers import Adam
  opt = Adam(lr=0.00006)
  model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['acc'])
  model.summary()
  plot_model(model, to_file='vgg.png')
  from keras.callbacks import ModelCheckpoint, EarlyStopping
  checkpoint = ModelCheckpoint("\content\vgg16_1.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
  early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=3, verbose=1, mode='auto')
  hist = model.fit_generator(steps_per_epoch=110,generator=traindata, validation_data= testdata, validation_steps=48,epochs=100,callbacks=[checkpoint,early])
  model.save("VGG16.h5")
